# Sentiment Analysis with an RNN

In this notebook, you'll implement a recurrent neural network that performs sentiment analysis. Using an RNN rather than a feedfoward network is more accurate since we can include information about the *sequence* of words. Here we'll use a dataset of movie reviews, accompanied by labels.

The architecture for this network is shown below.

<img src="assets/network_diagram.png" width=400px>

Here, we'll pass in words to an embedding layer. We need an embedding layer because we have tens of thousands of words, so we'll need a more efficient representation for our input data than one-hot encoded vectors. You should have seen this before from the word2vec lesson. You can actually train up an embedding with word2vec and use it here. But it's good enough to just have an embedding layer and let the network learn the embedding table on it's own.

From the embedding layer, the new representations will be passed to LSTM cells. These will add recurrent connections to the network so we can include information about the sequence of words in the data. Finally, the LSTM cells will go to a sigmoid output layer here. We're using the sigmoid because we're trying to predict if this text has positive or negative sentiment. The output layer will just be a single unit then, with a sigmoid activation function.

We don't care about the sigmoid outputs except for the very last one, we can ignore the rest. We'll calculate the cost from the output of the last step and the training label.

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
with open('./sentiment-network/reviews.txt', 'r') as f:
    reviews = f.read()
with open('./sentiment-network/labels.txt', 'r') as f:
    labels = f.read()

In [3]:
reviews[:2000]

'bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   \nstory of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is tu

## Data preprocessing

The first step when building a neural network model is getting your data into the proper form to feed into the network. Since we're using embedding layers, we'll need to encode each word with an integer. We'll also want to clean it up a bit.

You can see an example of the reviews data above. We'll want to get rid of those periods. Also, you might notice that the reviews are delimited with newlines `\n`. To deal with those, I'm going to split the text into each review using `\n` as the delimiter. Then I can combined all the reviews back together into one big string.

First, let's remove all punctuation. Then get all the text without the newlines and split it into individual words.

In [4]:
from string import punctuation
# Only keep not-punctuation characters
all_text = ''.join([c for c in reviews if c not in punctuation])
# Remove the newline '\n'
reviews = all_text.split('\n')

# Split the text into individual words
all_text = ' '.join(reviews)
words = all_text.split()

In [5]:
all_text[:2000]

'bromwell high is a cartoon comedy  it ran at the same time as some other programs about school life  such as  teachers   my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers   the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students  when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled          at           high  a classic line inspector i  m here to sack one of your teachers  student welcome to bromwell high  i expect that many adults of my age think that bromwell high is far fetched  what a pity that it isn  t    story of a man who has unnatural feelings for a pig  starts out with a opening scene that is a terrific example of absurd comedy  a formal orchestra audience is turned into an insane  violent m

In [6]:
words[:100]

['bromwell',
 'high',
 'is',
 'a',
 'cartoon',
 'comedy',
 'it',
 'ran',
 'at',
 'the',
 'same',
 'time',
 'as',
 'some',
 'other',
 'programs',
 'about',
 'school',
 'life',
 'such',
 'as',
 'teachers',
 'my',
 'years',
 'in',
 'the',
 'teaching',
 'profession',
 'lead',
 'me',
 'to',
 'believe',
 'that',
 'bromwell',
 'high',
 's',
 'satire',
 'is',
 'much',
 'closer',
 'to',
 'reality',
 'than',
 'is',
 'teachers',
 'the',
 'scramble',
 'to',
 'survive',
 'financially',
 'the',
 'insightful',
 'students',
 'who',
 'can',
 'see',
 'right',
 'through',
 'their',
 'pathetic',
 'teachers',
 'pomp',
 'the',
 'pettiness',
 'of',
 'the',
 'whole',
 'situation',
 'all',
 'remind',
 'me',
 'of',
 'the',
 'schools',
 'i',
 'knew',
 'and',
 'their',
 'students',
 'when',
 'i',
 'saw',
 'the',
 'episode',
 'in',
 'which',
 'a',
 'student',
 'repeatedly',
 'tried',
 'to',
 'burn',
 'down',
 'the',
 'school',
 'i',
 'immediately',
 'recalled',
 'at',
 'high']

### Encoding the words

The embedding lookup requires that we pass in integers to our network. The easiest way to do this is to create dictionaries that map the words in the vocabulary to integers. Then we can convert each of our reviews into integers so they can be passed into the network.

> **Exercise:** Now you're going to encode the words with integers. Build a dictionary that maps words to integers. Later we're going to pad our input vectors with zeros, so make sure the integers **start at 1, not 0**.
> Also, convert the reviews to integers and store the reviews in a new list called `reviews_ints`. 

In [7]:
# Create your dictionary that maps vocab words to integers here

from collections import Counter
# Create an unsorted counter for the list of words
counts = Counter(words)
# Sort the counter in descending order
vocab = sorted(counts, key = counts.get, reverse=True)
# Map vocab words to integers
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)} 

In [8]:
# Convert the reviews to integers, same shape as reviews list, but with integers
reviews_ints = []
for review in reviews:
    reviews_ints.append([vocab_to_int[word] for word in review.split()])

### Encoding the labels

Our labels are "positive" or "negative". To use these labels in our network, we need to convert them to 0 and 1.

> **Exercise:** Convert labels from `positive` and `negative` to 1 and 0, respectively.

In [9]:
# Convert labels to 1s and 0s for 'positive' and 'negative'
labels = labels.split('\n')
labels = np.array([1 if label == 'positive' else 0 for label in labels])

If you built `labels` correctly, you should see the next output.

In [10]:
from collections import Counter
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 1
Maximum review length: 2514


Okay, a couple issues here. We seem to have one review with zero length. And, the maximum review length is way too many steps for our RNN. Let's truncate to 200 steps. For reviews shorter than 200, we'll pad with 0s. For reviews longer than 200, we can truncate them to the first 200 characters.

> **Exercise:** First, remove the review with zero length from the `reviews_ints` list.

In [11]:
# Find reviews with non-zero length
non_zero_idx = [ii for ii, review in enumerate(reviews_ints) if len(review) != 0]
len(non_zero_idx)

25000

In [12]:
# Use the non_zero_idx to filter out that review with 0 length
reviews_ints = [reviews_ints[ii] for ii in non_zero_idx]
labels = np.array([labels[ii] for ii in non_zero_idx])

> **Exercise:** Now, create an array `features` that contains the data we'll pass to the network. The data should come from `review_ints`, since we want to feed integers to the network. Each row should be 200 elements long. For reviews shorter than 200 words, left pad with 0s. That is, if the review is `['best', 'movie', 'ever']`, `[117, 18, 128]` as integers, the row will look like `[0, 0, 0, ..., 0, 117, 18, 128]`. For reviews longer than 200, use on the first 200 words as the feature vector.

This isn't trivial and there are a bunch of ways to do this. But, if you're going to be building your own deep learning networks, you're going to have to get used to preparing your data.



In [13]:
seq_len = 200
# Create empty array features with 200 elements long
features = np.zeros((len(reviews_ints), seq_len), dtype=int)

# Pad the features array with reviews_ints
for i, row in enumerate(reviews_ints):
    features[i, -len(row):] = np.array(row)[:seq_len]

If you build features correctly, it should look like that cell output below.

In [14]:
features[:10,:100]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 21025,   308,     6,
            3,  1050,   207,     8,  2138,    32,     1,   171,    57,
           15,    49,    81,  5785,    44,   382,   110,   140,    15,
         5194,    60,   154,     9,     1,  4975,  5852,   475,    71,
            5,   260,    12, 21025,   308,    13,  1978,     6,    74,
         2395],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     

## Training, Validation, Test



With our data in nice shape, we'll split it into training, validation, and test sets.

> **Exercise:** Create the training, validation, and test sets here. You'll need to create sets for the features and the labels, `train_x` and `train_y` for example. Define a split fraction, `split_frac` as the fraction of data to keep in the training set. Usually this is set to 0.8 or 0.9. The rest of the data will be split in half to create the validation and testing data.

In [15]:
split_frac = 0.8 # fraction of data to keep in the training set

# Split the data into training and validation set
split_idx = int(len(features) * 0.8)
train_x, val_x = features[:split_idx], features[split_idx:]
train_y, val_y = labels[:split_idx], labels[split_idx:]

# Split half of the validation set as test set
test_idx = int(len(val_x) * 0.5)
val_x, test_x = val_x[:test_idx], val_x[test_idx:]
val_y, test_y = val_y[:test_idx], val_y[test_idx:]

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		(2500, 200)


With train, validation, and text fractions of 0.8, 0.1, 0.1, the final shapes should look like:
```
                    Feature Shapes:
Train set: 		 (20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		  (2500, 200)
```

## Build the graph

Here, we'll build the graph. First up, defining the hyperparameters.

* `lstm_size`: Number of units in the hidden layers in the LSTM cells. Usually larger is better performance wise. Common values are 128, 256, 512, etc.
* `lstm_layers`: Number of LSTM layers in the network. I'd start with 1, then add more if I'm underfitting.
* `batch_size`: The number of reviews to feed the network in one training pass. Typically this should be set as high as you can go without running out of memory.
* `learning_rate`: Learning rate

In [16]:
lstm_size = 256
lstm_layers = 1
batch_size = 128
learning_rate = 0.001

For the network itself, we'll be passing in our 200 element long review vectors. Each batch will be `batch_size` vectors. We'll also be using dropout on the LSTM layer, so we'll make a placeholder for the keep probability.

> **Exercise:** Create the `inputs_`, `labels_`, and drop out `keep_prob` placeholders using `tf.placeholder`.   
`labels_` needs to be two-dimensional to work with some functions later.  Since `keep_prob` is a scalar (a 0-dimensional tensor), you shouldn't provide a size to `tf.placeholder`.

In [17]:
n_words = len(vocab_to_int) + 1 # Adding 1 because we use 0's for padding, dictionary started at 1

# Create the graph object
graph = tf.Graph()
# Add nodes to the graph
with graph.as_default():
    inputs_ = tf.placeholder(tf.int32, [None, None], name = 'inputs')
    labels_ = tf.placeholder(tf.int32, [None, None], name = 'labels')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

### Embedding

Now we'll add an embedding layer. We need to do this because there are 74000 words in our vocabulary. It is massively inefficient to one-hot encode our classes here. You should remember dealing with this problem from the word2vec lesson. Instead of one-hot encoding, we can have an embedding layer and use that layer as a lookup table. You could train an embedding layer using word2vec, then load it here. But, it's fine to just make a new layer and let the network learn the weights.

> **Exercise:** Create the embedding lookup matrix as a `tf.Variable`. Use that embedding matrix to get the embedded vectors to pass to the LSTM cell with [`tf.nn.embedding_lookup`](https://www.tensorflow.org/api_docs/python/tf/nn/embedding_lookup). This function takes the embedding matrix and an input tensor, such as the review vectors. Then, it'll return another tensor with the embedded vectors. So, if the embedding layer has 200 units, the function will return a tensor with size [batch_size, 200].



In [18]:
# Size of the embedding vectors (number of units in the embedding layer)
embed_size = 300 

with graph.as_default():
    embedding = tf.Variable(tf.random_uniform((n_words, embed_size), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs_)

### LSTM cell

<img src="assets/network_diagram.png" width=400px>

Next, we'll create our LSTM cells to use in the recurrent network ([TensorFlow documentation](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn)). Here we are just defining what the cells look like. This isn't actually building the graph, just defining the type of cells we want in our graph.

To create a basic LSTM cell for the graph, you'll want to use `tf.contrib.rnn.BasicLSTMCell`. Looking at the function documentation:

```
tf.contrib.rnn.BasicLSTMCell(num_units, forget_bias=1.0, input_size=None, state_is_tuple=True, activation=<function tanh at 0x109f1ef28>)
```

you can see it takes a parameter called `num_units`, the number of units in the cell, called `lstm_size` in this code. So then, you can write something like 

```
lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
```

to create an LSTM cell with `num_units`. Next, you can add dropout to the cell with `tf.contrib.rnn.DropoutWrapper`. This just wraps the cell in another cell, but with dropout added to the inputs and/or outputs. It's a really convenient way to make your network better with almost no effort! So you'd do something like

```
drop = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
```

Most of the time, your network will have better performance with more layers. That's sort of the magic of deep learning, adding more layers allows the network to learn really complex relationships. Again, there is a simple way to create multiple layers of LSTM cells with `tf.contrib.rnn.MultiRNNCell`:

```
cell = tf.contrib.rnn.MultiRNNCell([drop] * lstm_layers)
```

Here, `[drop] * lstm_layers` creates a list of cells (`drop`) that is `lstm_layers` long. The `MultiRNNCell` wrapper builds this into multiple layers of RNN cells, one for each cell in the list.

So the final cell you're using in the network is actually multiple (or just one) LSTM cells with dropout. But it all works the same from an achitectural viewpoint, just a more complicated graph in the cell.

> **Exercise:** Below, use `tf.contrib.rnn.BasicLSTMCell` to create an LSTM cell. Then, add drop out to it with `tf.contrib.rnn.DropoutWrapper`. Finally, create multiple LSTM layers with `tf.contrib.rnn.MultiRNNCell`.

Here is [a tutorial on building RNNs](https://www.tensorflow.org/tutorials/recurrent) that will help you out.


In [19]:
with graph.as_default():
    # Your basic LSTM cell
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
    # Add dropout to the cell
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([drop] * lstm_layers)
    
    # Getting an initial state of all zeros
    initial_state = cell.zero_state(batch_size, tf.float32)

### RNN forward pass

<img src="assets/network_diagram.png" width=400px>

Now we need to actually run the data through the RNN nodes. You can use [`tf.nn.dynamic_rnn`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn) to do this. You'd pass in the RNN cell you created (our multiple layered LSTM `cell` for instance), and the inputs to the network.

```
outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, initial_state=initial_state)
```

Above I created an initial state, `initial_state`, to pass to the RNN. This is the cell state that is passed between the hidden layers in successive time steps. `tf.nn.dynamic_rnn` takes care of most of the work for us. We pass in our cell and the input to the cell, then it does the unrolling and everything else for us. It returns outputs for each time step and the final_state of the hidden layer.

> **Exercise:** Use `tf.nn.dynamic_rnn` to add the forward pass through the RNN. Remember that we're actually passing in vectors from the embedding layer, `embed`.



In [20]:
with graph.as_default():
    outputs, final_state = tf.nn.dynamic_rnn(cell, embed,
                                             initial_state = initial_state)

### Output

We only care about the final output, we'll be using that as our sentiment prediction. So we need to grab the last output with `outputs[:, -1]`, the calculate the cost from that and `labels_`.

In [21]:
with graph.as_default():
    predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn = tf.sigmoid)
    cost = tf.losses.mean_squared_error(labels_, predictions)
    
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

### Validation accuracy

Here we can add a few nodes to calculate the accuracy which we'll use in the validation pass.

In [22]:
with graph.as_default():
    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

### Batching

This is a simple function for returning batches from our data. First it removes data such that we only have full batches. Then it iterates through the `x` and `y` arrays and returns slices out of those arrays with size `[batch_size]`.

In [23]:
def get_batches(x, y, batch_size = 100):
    
    n_batches = len(x)//batch_size
    x, y = x[:n_batches * batch_size], y[:n_batches * batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii  +batch_size], y[ii:ii + batch_size]

## Training

Below is the typical training code. If you want to do this yourself, feel free to delete all this code and implement it yourself. Before you run this, make sure the `checkpoints` directory exists.

In [27]:
epochs = 10

with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state)
        
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            feed = {inputs_: x,
                    labels_: y[:, None],
                    keep_prob: 0.5,
                    initial_state: state}
            loss, state, _ = sess.run([cost, final_state, optimizer], feed_dict=feed)
            
            if iteration%1==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))

            if iteration%25==0:
                val_acc = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for x, y in get_batches(val_x, val_y, batch_size):
                    feed = {inputs_: x,
                            labels_: y[:, None],
                            keep_prob: 1,
                            initial_state: val_state}
                    batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)
                    val_acc.append(batch_acc)
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
            iteration +=1
    saver.save(sess, "checkpoints/sentiment.ckpt")

Epoch: 0/10 Iteration: 1 Train loss: 0.245
Epoch: 0/10 Iteration: 2 Train loss: 0.247
Epoch: 0/10 Iteration: 3 Train loss: 0.261
Epoch: 0/10 Iteration: 4 Train loss: 0.252
Epoch: 0/10 Iteration: 5 Train loss: 0.249
Epoch: 0/10 Iteration: 6 Train loss: 0.251
Epoch: 0/10 Iteration: 7 Train loss: 0.246
Epoch: 0/10 Iteration: 8 Train loss: 0.249
Epoch: 0/10 Iteration: 9 Train loss: 0.250
Epoch: 0/10 Iteration: 10 Train loss: 0.262
Epoch: 0/10 Iteration: 11 Train loss: 0.248
Epoch: 0/10 Iteration: 12 Train loss: 0.232
Epoch: 0/10 Iteration: 13 Train loss: 0.251
Epoch: 0/10 Iteration: 14 Train loss: 0.231
Epoch: 0/10 Iteration: 15 Train loss: 0.234
Epoch: 0/10 Iteration: 16 Train loss: 0.238
Epoch: 0/10 Iteration: 17 Train loss: 0.228
Epoch: 0/10 Iteration: 18 Train loss: 0.222
Epoch: 0/10 Iteration: 19 Train loss: 0.241
Epoch: 0/10 Iteration: 20 Train loss: 0.208
Epoch: 0/10 Iteration: 21 Train loss: 0.205
Epoch: 0/10 Iteration: 22 Train loss: 0.201
Epoch: 0/10 Iteration: 23 Train loss: 0.2

Epoch: 1/10 Iteration: 184 Train loss: 0.199
Epoch: 1/10 Iteration: 185 Train loss: 0.183
Epoch: 1/10 Iteration: 186 Train loss: 0.217
Epoch: 1/10 Iteration: 187 Train loss: 0.232
Epoch: 1/10 Iteration: 188 Train loss: 0.248
Epoch: 1/10 Iteration: 189 Train loss: 0.214
Epoch: 1/10 Iteration: 190 Train loss: 0.229
Epoch: 1/10 Iteration: 191 Train loss: 0.239
Epoch: 1/10 Iteration: 192 Train loss: 0.221
Epoch: 1/10 Iteration: 193 Train loss: 0.230
Epoch: 1/10 Iteration: 194 Train loss: 0.238
Epoch: 1/10 Iteration: 195 Train loss: 0.247
Epoch: 1/10 Iteration: 196 Train loss: 0.217
Epoch: 1/10 Iteration: 197 Train loss: 0.213
Epoch: 1/10 Iteration: 198 Train loss: 0.188
Epoch: 1/10 Iteration: 199 Train loss: 0.213
Epoch: 1/10 Iteration: 200 Train loss: 0.218
Val acc: 0.627
Epoch: 1/10 Iteration: 201 Train loss: 0.229
Epoch: 1/10 Iteration: 202 Train loss: 0.218
Epoch: 1/10 Iteration: 203 Train loss: 0.231
Epoch: 1/10 Iteration: 204 Train loss: 0.216
Epoch: 1/10 Iteration: 205 Train loss: 0

Epoch: 2/10 Iteration: 364 Train loss: 0.140
Epoch: 2/10 Iteration: 365 Train loss: 0.181
Epoch: 2/10 Iteration: 366 Train loss: 0.167
Epoch: 2/10 Iteration: 367 Train loss: 0.131
Epoch: 2/10 Iteration: 368 Train loss: 0.195
Epoch: 2/10 Iteration: 369 Train loss: 0.157
Epoch: 2/10 Iteration: 370 Train loss: 0.204
Epoch: 2/10 Iteration: 371 Train loss: 0.157
Epoch: 2/10 Iteration: 372 Train loss: 0.165
Epoch: 2/10 Iteration: 373 Train loss: 0.194
Epoch: 2/10 Iteration: 374 Train loss: 0.140
Epoch: 2/10 Iteration: 375 Train loss: 0.182
Val acc: 0.660
Epoch: 2/10 Iteration: 376 Train loss: 0.252
Epoch: 2/10 Iteration: 377 Train loss: 0.314
Epoch: 2/10 Iteration: 378 Train loss: 0.337
Epoch: 2/10 Iteration: 379 Train loss: 0.327
Epoch: 2/10 Iteration: 380 Train loss: 0.345
Epoch: 2/10 Iteration: 381 Train loss: 0.321
Epoch: 2/10 Iteration: 382 Train loss: 0.283
Epoch: 2/10 Iteration: 383 Train loss: 0.274
Epoch: 2/10 Iteration: 384 Train loss: 0.246
Epoch: 2/10 Iteration: 385 Train loss: 0

Epoch: 3/10 Iteration: 544 Train loss: 0.146
Epoch: 3/10 Iteration: 545 Train loss: 0.164
Epoch: 3/10 Iteration: 546 Train loss: 0.134
Epoch: 3/10 Iteration: 547 Train loss: 0.133
Epoch: 3/10 Iteration: 548 Train loss: 0.142
Epoch: 3/10 Iteration: 549 Train loss: 0.221
Epoch: 3/10 Iteration: 550 Train loss: 0.144
Val acc: 0.723
Epoch: 3/10 Iteration: 551 Train loss: 0.157
Epoch: 3/10 Iteration: 552 Train loss: 0.197
Epoch: 3/10 Iteration: 553 Train loss: 0.217
Epoch: 3/10 Iteration: 554 Train loss: 0.185
Epoch: 3/10 Iteration: 555 Train loss: 0.139
Epoch: 3/10 Iteration: 556 Train loss: 0.114
Epoch: 3/10 Iteration: 557 Train loss: 0.156
Epoch: 3/10 Iteration: 558 Train loss: 0.114
Epoch: 3/10 Iteration: 559 Train loss: 0.133
Epoch: 3/10 Iteration: 560 Train loss: 0.183
Epoch: 3/10 Iteration: 561 Train loss: 0.190
Epoch: 3/10 Iteration: 562 Train loss: 0.144
Epoch: 3/10 Iteration: 563 Train loss: 0.180
Epoch: 3/10 Iteration: 564 Train loss: 0.124
Epoch: 3/10 Iteration: 565 Train loss: 0

Epoch: 4/10 Iteration: 724 Train loss: 0.069
Epoch: 4/10 Iteration: 725 Train loss: 0.101
Val acc: 0.823
Epoch: 4/10 Iteration: 726 Train loss: 0.093
Epoch: 4/10 Iteration: 727 Train loss: 0.085
Epoch: 4/10 Iteration: 728 Train loss: 0.067
Epoch: 4/10 Iteration: 729 Train loss: 0.065
Epoch: 4/10 Iteration: 730 Train loss: 0.095
Epoch: 4/10 Iteration: 731 Train loss: 0.101
Epoch: 4/10 Iteration: 732 Train loss: 0.095
Epoch: 4/10 Iteration: 733 Train loss: 0.056
Epoch: 4/10 Iteration: 734 Train loss: 0.081
Epoch: 4/10 Iteration: 735 Train loss: 0.078
Epoch: 4/10 Iteration: 736 Train loss: 0.069
Epoch: 4/10 Iteration: 737 Train loss: 0.135
Epoch: 4/10 Iteration: 738 Train loss: 0.096
Epoch: 4/10 Iteration: 739 Train loss: 0.093
Epoch: 4/10 Iteration: 740 Train loss: 0.076
Epoch: 4/10 Iteration: 741 Train loss: 0.074
Epoch: 4/10 Iteration: 742 Train loss: 0.077
Epoch: 4/10 Iteration: 743 Train loss: 0.082
Epoch: 4/10 Iteration: 744 Train loss: 0.051
Epoch: 4/10 Iteration: 745 Train loss: 0

Epoch: 5/10 Iteration: 904 Train loss: 0.000
Epoch: 5/10 Iteration: 905 Train loss: 0.000
Epoch: 5/10 Iteration: 906 Train loss: 0.000
Epoch: 5/10 Iteration: 907 Train loss: 0.000
Epoch: 5/10 Iteration: 908 Train loss: 0.000
Epoch: 5/10 Iteration: 909 Train loss: 0.000
Epoch: 5/10 Iteration: 910 Train loss: 0.000
Epoch: 5/10 Iteration: 911 Train loss: 0.000
Epoch: 5/10 Iteration: 912 Train loss: 0.000
Epoch: 5/10 Iteration: 913 Train loss: 0.000
Epoch: 5/10 Iteration: 914 Train loss: 0.000
Epoch: 5/10 Iteration: 915 Train loss: 0.000
Epoch: 5/10 Iteration: 916 Train loss: 0.000
Epoch: 5/10 Iteration: 917 Train loss: 0.000
Epoch: 5/10 Iteration: 918 Train loss: 0.000
Epoch: 5/10 Iteration: 919 Train loss: 0.000
Epoch: 5/10 Iteration: 920 Train loss: 0.000
Epoch: 5/10 Iteration: 921 Train loss: 0.000
Epoch: 5/10 Iteration: 922 Train loss: 0.000
Epoch: 5/10 Iteration: 923 Train loss: 0.000
Epoch: 5/10 Iteration: 924 Train loss: 0.000
Epoch: 5/10 Iteration: 925 Train loss: 0.000
Val acc: 0

Epoch: 6/10 Iteration: 1082 Train loss: 0.000
Epoch: 6/10 Iteration: 1083 Train loss: 0.000
Epoch: 6/10 Iteration: 1084 Train loss: 0.000
Epoch: 6/10 Iteration: 1085 Train loss: 0.000
Epoch: 6/10 Iteration: 1086 Train loss: 0.000
Epoch: 6/10 Iteration: 1087 Train loss: 0.000
Epoch: 6/10 Iteration: 1088 Train loss: 0.000
Epoch: 6/10 Iteration: 1089 Train loss: 0.000
Epoch: 6/10 Iteration: 1090 Train loss: 0.000
Epoch: 6/10 Iteration: 1091 Train loss: 0.000
Epoch: 6/10 Iteration: 1092 Train loss: 0.000
Epoch: 7/10 Iteration: 1093 Train loss: 0.246
Epoch: 7/10 Iteration: 1094 Train loss: 0.151
Epoch: 7/10 Iteration: 1095 Train loss: 0.122
Epoch: 7/10 Iteration: 1096 Train loss: 0.089
Epoch: 7/10 Iteration: 1097 Train loss: 0.055
Epoch: 7/10 Iteration: 1098 Train loss: 0.031
Epoch: 7/10 Iteration: 1099 Train loss: 0.045
Epoch: 7/10 Iteration: 1100 Train loss: 0.130
Val acc: 0.505
Epoch: 7/10 Iteration: 1101 Train loss: 0.238
Epoch: 7/10 Iteration: 1102 Train loss: 0.372
Epoch: 7/10 Iterati

Epoch: 8/10 Iteration: 1258 Train loss: 0.000
Epoch: 8/10 Iteration: 1259 Train loss: 0.000
Epoch: 8/10 Iteration: 1260 Train loss: 0.000
Epoch: 8/10 Iteration: 1261 Train loss: 0.000
Epoch: 8/10 Iteration: 1262 Train loss: 0.002
Epoch: 8/10 Iteration: 1263 Train loss: 0.000
Epoch: 8/10 Iteration: 1264 Train loss: 0.008
Epoch: 8/10 Iteration: 1265 Train loss: 0.008
Epoch: 8/10 Iteration: 1266 Train loss: 0.008
Epoch: 8/10 Iteration: 1267 Train loss: 0.007
Epoch: 8/10 Iteration: 1268 Train loss: 0.008
Epoch: 8/10 Iteration: 1269 Train loss: 0.007
Epoch: 8/10 Iteration: 1270 Train loss: 0.006
Epoch: 8/10 Iteration: 1271 Train loss: 0.000
Epoch: 8/10 Iteration: 1272 Train loss: 0.008
Epoch: 8/10 Iteration: 1273 Train loss: 0.000
Epoch: 8/10 Iteration: 1274 Train loss: 0.000
Epoch: 8/10 Iteration: 1275 Train loss: 0.000
Val acc: 0.889
Epoch: 8/10 Iteration: 1276 Train loss: 0.001
Epoch: 8/10 Iteration: 1277 Train loss: 0.000
Epoch: 8/10 Iteration: 1278 Train loss: 0.001
Epoch: 8/10 Iterati

Epoch: 9/10 Iteration: 1434 Train loss: 0.000
Epoch: 9/10 Iteration: 1435 Train loss: 0.002
Epoch: 9/10 Iteration: 1436 Train loss: 0.000
Epoch: 9/10 Iteration: 1437 Train loss: 0.000
Epoch: 9/10 Iteration: 1438 Train loss: 0.000
Epoch: 9/10 Iteration: 1439 Train loss: 0.000
Epoch: 9/10 Iteration: 1440 Train loss: 0.000
Epoch: 9/10 Iteration: 1441 Train loss: 0.000
Epoch: 9/10 Iteration: 1442 Train loss: 0.000
Epoch: 9/10 Iteration: 1443 Train loss: 0.002
Epoch: 9/10 Iteration: 1444 Train loss: 0.000
Epoch: 9/10 Iteration: 1445 Train loss: 0.000
Epoch: 9/10 Iteration: 1446 Train loss: 0.000
Epoch: 9/10 Iteration: 1447 Train loss: 0.000
Epoch: 9/10 Iteration: 1448 Train loss: 0.000
Epoch: 9/10 Iteration: 1449 Train loss: 0.000
Epoch: 9/10 Iteration: 1450 Train loss: 0.000
Val acc: 0.967
Epoch: 9/10 Iteration: 1451 Train loss: 0.000
Epoch: 9/10 Iteration: 1452 Train loss: 0.000
Epoch: 9/10 Iteration: 1453 Train loss: 0.000
Epoch: 9/10 Iteration: 1454 Train loss: 0.000
Epoch: 9/10 Iterati

## Testing

In [28]:
test_acc = []
with tf.Session(graph=graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    for ii, (x, y) in enumerate(get_batches(test_x, test_y, batch_size), 1):
        feed = {inputs_: x,
                labels_: y[:, None],
                keep_prob: 1,
                initial_state: test_state}
        batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
        test_acc.append(batch_acc)
    print("Test accuracy: {:.3f}".format(np.mean(test_acc)))

INFO:tensorflow:Restoring parameters from checkpoints/sentiment.ckpt
Test accuracy: 0.962
